In [2]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(dirname(pwd()) * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

┌ Warning: Replacing docs for `Main.WaterRegulation.connect_reservoirs :: Tuple{Reservoir, Array{Reservoir}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.find_us_reservoir :: Tuple{Reservoir}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243


┌ Warning: Replacing docs for `Main.WaterRegulation.OtherParticipant :: Tuple{Array{Participant}, Participant, Array{Reservoir}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.calculate_balance :: Tuple{Dict{Reservoir, Float64}, Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Reservoir}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243


┌ Warning: Replacing docs for `Main.WaterRegulation.adjust_flow! :: Union{Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{HydropowerPlant, Float64}}, Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{HydropowerPlant, Float64}, Any}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.update_reservoir :: Tuple{Dict{Reservoir, Float64}, Int64}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.update_ind_reservoir :: Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{Reservoir, Float64}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.read_data :: Tuple{String}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs f

┌ Warning: Replacing docs for `Main.WaterRegulation.Calculate_POver :: Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{HydropowerPlant, Float64}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.power_swap :: Tuple{Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Dict{Reservoir, Float64}, Dict{Participant, Dict{HydropowerPlant, Float64}}, Dict{HydropowerPlant, Float64}, Dict{HydropowerPlant, Float64}, Array{Participant}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.PS_Reduction :: Tuple{Dict{HydropowerPlant, Float64}, Any, Any, Array{Participant}, Any}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.calculate_pr

┌ Warning: Replacing docs for `Main.WaterRegulation._collect_solution_Short :: Tuple{Any, Any, Any}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243
┌ Warning: Replacing docs for `Main.WaterRegulation.RealTimeBalancing :: Tuple{Vector{Reservoir}, Participant, Dict{Reservoir, Float64}, Dict{Reservoir, Float64}, Dict{Int64, Float64}}` in module `Main.WaterRegulation`
└ @ Base.Docs docs\Docs.jl:243


### Simulations of Water Regulation Procedure

We would like to set up a skeleton for the day-to-day decision making of hydropower producers and the Water Regulation Company. The main steps of the Water Regulation company consist of:

* Determining a Reference Flow for the next nomination period (Inflow Forecasting)
* Receiving pre-bid nominations and calculating adjusted flow, power swaps, etc. -> Send these parameters back to the participants
* Recalculate based on renominations by power producers after bidding

For the Power Producers / Participants:  

* Fetch and Calculate all Parameters for the Water Regulation Procedure (Individual Reservoir Level etc.)
* Price and Inflow Forecasting (Fix Price Points)
* Medium-Term Hydropower Model Calculation, obtain Water Value Cuts
* Decision Model 1: Bidding and Pre-Market Clearing Nomination
* Decision Model 2: Short-Term Optimization and Renomination
* Decision Model 3: Fixed Water Flow scheduling and real time balancing

This constitutes one round of simulation.


In [4]:
filepath_Ljungan = dirname(pwd()) * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = dirname(pwd()) *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = dirname(pwd()) * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)

println("The reservoir system consists of $([r.dischargepoint for r in R])")
println("Downstream of the reservoirs we have power plants $([k.name for k in K])")
for j in J
    println("$(j.name) owns $(j.plants), which constitutes a participation rate of: \n ", j.participationrate)
end


The reservoir system consists of ["Flasjon", "Holsmjon"]
Downstream of the reservoirs we have power plants ["Flasjo", "Trangfors", "Ratan", "Turinge", "Bursnas", "Jarnvagsforsen", "Parteboda", "Hermansboda", "Ljunga", "Nederede", "Skallbole", "Matfors", "Viforsen"]
Sydkraft owns HydropowerPlant[Flasjo, Trangfors, Ratan, Turinge, Bursnas], which constitutes a participation rate of: 
 ________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

Fortum owns HydropowerPlant[Parteboda, Hermansboda, Ljunga], which constitutes a participation rate of: 
 ________________________________
Flasjon  | 0.8200000000000001
Holsmjon | 0.8200000000000001

Statkraft owns HydropowerPlant[Nederede, Skallbole, Matfors, Viforsen, Jarnvagsforsen], which constitutes a participation rate of: 
 ________________________________
Flasjon  | 1.86    
Holsmjon | 1.86    



#### Load Data

Other than the river system, the relevant data for optimization is organized in Dataframes.
We load in cleaned versions of the data.

In [5]:
filepath_prices = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Spot Prices\\prices_df.csv"
filepath_inflows = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Inflow\\Data from Flasjoen and Holmsjoen.csv"

price_data = prepare_pricedata(filepath_prices)
inflow_data = prepare_inflowdata(filepath_inflows)

first(inflow_data, 10)

Row,Date,Flasjon Water Level,Flasjon Volume,Flasjon Inflow,Flasjon discharge,Holmsjon Water Level,Holmsjon Volume,Holmsjon Inflow,Holmsjon Discharge,CalendarWeek,year
,Date,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Int64,Int64
1,2006-01-01,484.0,2087.0,12.8,49.0,244.39,1928,13.6,53.2,52,2006
2,2006-01-02,483.83,2056.0,13.6,49.0,244.42,1947,22.7,89.1,1,2006
3,2006-01-03,483.69,2031.0,10.2,49.0,244.41,1941,13.4,85.9,1,2006
4,2006-01-04,483.52,2000.0,10.2,50.0,244.41,1941,23.0,88.1,1,2006
5,2006-01-05,483.34,1968.0,8.2,52.0,244.41,1941,15.0,86.2,1,2006
6,2006-01-06,483.14,1932.0,7.9,52.0,244.4,1934,19.1,74.2,1,2006
7,2006-01-07,482.94,1896.0,6.2,53.0,244.41,1941,16.0,84.2,1,2006
8,2006-01-08,482.73,1860.0,11.2,53.0,244.4,1934,22.0,85.2,1,2006
9,2006-01-09,482.53,1826.0,2.6,53.0,244.39,1928,-0.3,104.2,2,2006


### Price Forecasting

Price is sensitive to various factors. Daytime, Weekday and time of year play a relevant role in how prices usually behave.

* For the bidding and short-term optimization problems, part of the uncertainty lies in unknown hourly prices.

* For the medium term problem, we are interested in daily (average) prices.

* We want to obtain Price Points which will be important in the analysis from the historic data.

* We need penalty parameters for using the balancing markets, which could be obtained from using maximum / minimum values fromm observed spot market prices / generated scenarios.

In [6]:
const scenario_count_prices_hourly = 10
const scenario_count_prices_weekly = 2
const stage_count_short = 8
const price_point_count = 5

PriceScenariosMedium = Price_Scenarios_Weekly(price_data, scenario_count_prices_weekly)
PriceScenariosShort = Price_Scenarios_Hourly(price_data, scenario_count_prices_hourly)
PPoints = Create_Price_Points(price_data, 5)
mu_up, mu_down = BalanceParameters(PriceScenariosShort)

(93.08, 5.0)

### Inflow Forecasting

Inflow has extreme seasonal differences, depending on geographical location.  
For the Ljungan River System we could observe that most inflow occurs during the spring melt around May.

In [7]:
const ColumnReservoir = Dict{Reservoir, String}(R[1] => "Flasjon Inflow", R[2] => "Holmsjon Inflow")
const scenario_count_inflows_weekly = 2
const scenario_count_inflows_short = 1


InflowScenariosShort = Inflow_Scenarios_Short(inflow_data, 1, ColumnReservoir, R)
InflowScenariosMedium = Inflow_Scenarios_Weekly(inflow_data, ColumnReservoir, scenario_count_inflows_weekly, R)
first(InflowScenariosShort, 10)

8-element Vector{Pair{Int64, Dict{Reservoir, Vector{Float64}}}}:
 5 => Dict(Flasjon => [6.58], Holsmjon => [24.2])
 4 => Dict(Flasjon => [10.43], Holsmjon => [38.9])
 6 => Dict(Flasjon => [8.2], Holsmjon => [0.0])
 7 => Dict(Flasjon => [15.23], Holsmjon => [23.5])
 2 => Dict(Flasjon => [11.02], Holsmjon => [2.7])
 8 => Dict(Flasjon => [15.23], Holsmjon => [0.0])
 3 => Dict(Flasjon => [6.6], Holsmjon => [43.2])
 1 => Dict(Flasjon => [13.49], Holsmjon => [3.0])

### Create Uncertainty Set

Depending on the Problem, we create the uncertainty sets from both price and inflow Scenarios.
This becomes more difficult for the Anticipatory Bidding Problem.

In [8]:
const stage_count_bidding = 3
Ω_medium, P_medium =  create_Ω_medium(PriceScenariosMedium, InflowScenariosMedium, R);
Ω_NA, P_NA = create_Ω_Nonanticipatory(PriceScenariosShort, InflowScenariosShort, R)
Ω_NA[1]

"""
    create_Ω_Anticipatory(PriceScenariosShort, InflowScenariosShort, j, R)

    create Uncertainty Set for the Anticipatory Problem: 
    Addiitionally to the Nonanticipatory Case, we associate deterministic sets of inflows and prices with other producers' nominations.
    The Uncertainty set is extended entrywise by the others' nomination. 
"""

function create_Ω_Anticipatory(PriceScenariosShort, InflowScenariosShort, OthersNominations, j::Participant, R::Vector{Reservoir}, stage_count = stage_count_bidding)
    Ω = Dict(i => [(inflow = Dict(r => InflowScenariosShort[i][r][j] for r in R), price = c) for c in PriceScenariosShort[i] for j in eachindex(InflowScenariosShort[i][R[1]])] for i in 1:stage_count)
    P = Dict(s => [1/length(eachindex(Ω[s])) for i in eachindex(Ω[s])] for s in 1:stage_count)
    return Ω, P
end
# Ω_A, P_A = create_Ω_Anticipatory()

create_Ω_Anticipatory (generic function with 2 methods)

### Reference Flow

To calculate a reference flow for each reservoir, the Water Regulation Company has to combine information from inflow, historical reservoir trajectories and current reservoir level.  

In [39]:
l_traj, f = AverageReservoirLevel(R, inflow_data)
Qref = CalculateReferenceFlow(R, l_traj, f, 2)

________________________________
Flasjon  | 2739.9285714285716
Holsmjon | 1492.2698412698414
________________________________
Flasjon  | 8.683968253968256
Holsmjon | 18.135714285714283



Dict{Reservoir, Float64} with 2 entries:
  Flasjon  => -97.0201
  Holsmjon => -80.76

## Participants

Every Participant individually plans their bidding and production. Irrespective of the strategy, only limited information is available. Internal to the participants problems are 

* Price Forecasts
* Nomination to Water Regulation Company
* Bidding at Electricity Market

### Medium Term Problem

The Medium Term / Seasonal Optimization Problem is necessary to obtain Cuts for the Water Value Function.

Therefor the point of a functioin containing this decision model is to return cut coefficients of the Water Value Function.

* What are the inputs to the Medium Term problem?
* How are the cuts calculated?
* How many cuts should be calculated?

In [8]:
"""
ReservoirLevelCuts(R)
    Create an Array of reservoir values used in the cut generation of the Water Value Function.
    These Values should correspond to relevant values in the proximity of the current reservoir level. Examples:
    - No Change (Current Reservoir Level)
    - Low/High Inflow
    - No/High production
    These all give us cuts to estimate an area which is relevant to the next stages optimization.
"""
const iterations = 500


function ReservoirLevelCuts(R::Vector{Reservoir}, K::Vector{HydropowerPlant})::Dict{Int64, Dict{Reservoir, Float64}}
    reservoircutvalues = Dict(r => [r.currentvolume, 0.0, r.currentvolume/2] for r in R)
    combs = Iterators.product(values(reservoircutvalues)...)
    cuts = Dict(i => Dict(r => v for (r,v) in zip(keys(reservoircutvalues), combo)) for (i, combo) in enumerate(combs))
    print(values(cuts))
    return cuts
end

model_medium, V_medium = MediumTermModel(R, K, Ω_medium, P_medium, stage_count, iterations)
WaterValueCuts = WaterValueCutGeneration(V_medium, cuts)


UndefVarError: UndefVarError: `Ω_medium` not defined

### Bidding Problem


### Short Term Optimization

### Real Time Balancing and Scheduling

# Simulation

For a preset amount of rounds, we simulate the reservoir system through the functions and structures as described above. It is important to have a way to save results adequately! This encapsulates the calculations, and evaluations and visualizations can take place at a later point in time.